In [119]:
import pandas as pd

X_train = '/home/wjunneng/Ubuntu/2019-CCF-Prediction-Of-The-Quality-Compliance-Rate-Of-Workpieces-In-Discrete-Manufacturing-Processes/data/cache/X_train_smote.h5'
y_train = '/home/wjunneng/Ubuntu/2019-CCF-Prediction-Of-The-Quality-Compliance-Rate-Of-Workpieces-In-Discrete-Manufacturing-Processes/data/cache/y_train_smote.h5'
X_test = '/home/wjunneng/Ubuntu/2019-CCF-Prediction-Of-The-Quality-Compliance-Rate-Of-Workpieces-In-Discrete-Manufacturing-Processes/data/cache/X_test.h5'

X_train = pd.read_hdf(path_or_buf=X_train, mode='r', key='X_train')
y_train = pd.read_hdf(path_or_buf=y_train, mode='r', key='y_train')
X_test = pd.read_hdf(path_or_buf=X_test, mode='r', key='X_test')

columns = ['Parameter1', 'Parameter2', 'Parameter3', 'Parameter4', 'Parameter5', 'Parameter6',
                          'Parameter7', 'Parameter8', 'Parameter9', 'Parameter10']

categorical_columns = ['Parameter5', 'Parameter6',
                          'Parameter7', 'Parameter8', 'Parameter9', 'Parameter10']

X_train = X_train[columns]
X_test = X_test[columns]


In [120]:
X_train.head()


,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10
0,-1.006836,-0.530762,1.590820,1.132812,0.000421,0.000612,2286.0,0.035407,0.593081,1.010742
1,-0.130493,-1.026367,-1.021484,1.210938,0.000909,0.002398,2286.0,0.035407,0.593081,1.010742
2,-0.908203,1.412109,-0.968750,-0.900391,0.000909,0.001972,2286.0,0.035407,0.593081,1.010742
3,0.951660,-0.988770,-1.043945,-0.331055,0.000909,0.002398,2286.0,0.035407,0.593081,1.010742
4,-0.995605,-1.034180,-1.059570,1.620117,0.000909,0.002398,2286.0,0.035407,0.593081,1.010742


In [121]:
X_train.head()


,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10
0,-1.006836,-0.530762,1.590820,1.132812,0.000421,0.000612,2286.0,0.035407,0.593081,1.010742
1,-0.130493,-1.026367,-1.021484,1.210938,0.000909,0.002398,2286.0,0.035407,0.593081,1.010742
2,-0.908203,1.412109,-0.968750,-0.900391,0.000909,0.001972,2286.0,0.035407,0.593081,1.010742
3,0.951660,-0.988770,-1.043945,-0.331055,0.000909,0.002398,2286.0,0.035407,0.593081,1.010742
4,-0.995605,-1.034180,-1.059570,1.620117,0.000909,0.002398,2286.0,0.035407,0.593081,1.010742


In [122]:
y_train.head()


0    2
1    3
2    3
3    3
4    3
dtype: int64

In [123]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb


X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

X_train['Quality_label'] = y_train
X_valid['Quality_label'] = y_valid
X_train['Is_Test'] = 0
X_valid['Is_Test'] = 1

# 将 Train 和 Test 合成一个数据集。Quality_label是数据本来的Y，所以剔除。
df_adv = pd.concat([X_train, X_valid])

adv_data = lgb.Dataset(data=df_adv.drop('Is_Test', axis=1), label=df_adv.loc[:, 'Is_Test'])

# 定义模型参数
params = {
    'boosting_type': 'gbdt',
    'colsample_bytree': 1,
    'learning_rate': 0.1,
    'max_depth': 5,
    'min_child_samples': 100,
    'min_child_weight': 1,
    'min_split_gain': 0.0,
    'num_leaves': 20,
    'objective': 'binary',
    'random_state': 50,
    'subsample': 1.0,
    'subsample_freq': 0,
    'metric': 'auc',
    'num_threads': 8
}

# 交叉验证
adv_cv_results = lgb.cv(
    params,
    adv_data,
    num_boost_round=10000,
    nfold=5,
    categorical_feature=categorical_columns,
    early_stopping_rounds=200,
    verbose_eval=True,
    seed=42)

print('交叉验证中最优的AUC为 {:.5f}，对应的标准差为{:.5f}.'.format(
    adv_cv_results['auc-mean'][-1], adv_cv_results['auc-stdv'][-1]))

print('模型最优的迭代次数为{}.'.format(len(adv_cv_results['auc-mean'])))


[1]	cv_agg's auc: 0.513567 + 0.0154579
[2]	cv_agg's auc: 0.516969 + 0.0161124
[3]	cv_agg's auc: 0.519606 + 0.01231
[4]	cv_agg's auc: 0.517641 + 0.0109929
[5]	cv_agg's auc: 0.516329 + 0.0105724
[6]	cv_agg's auc: 0.512623 + 0.0129872
[7]	cv_agg's auc: 0.511112 + 0.0116579
[8]	cv_agg's auc: 0.512458 + 0.00776031
[9]	cv_agg's auc: 0.510481 + 0.00717166
[10]	cv_agg's auc: 0.511954 + 0.00799246
[11]	cv_agg's auc: 0.512623 + 0.0076673
[12]	cv_agg's auc: 0.512033 + 0.00651754
[13]	cv_agg's auc: 0.513016 + 0.00759083
[14]	cv_agg's auc: 0.514885 + 0.00965241
[15]	cv_agg's auc: 0.515002 + 0.0099335
[16]	cv_agg's auc: 0.514192 + 0.0108676
[17]	cv_agg's auc: 0.514421 + 0.0111697
[18]	cv_agg's auc: 0.514412 + 0.0115082
[19]	cv_agg's auc: 0.513005 + 0.0107982
[20]	cv_agg's auc: 0.513469 + 0.00986755
[21]	cv_agg's auc: 0.513408 + 0.010231
[22]	cv_agg's auc: 0.511039 + 0.0104796
[23]	cv_agg's auc: 0.510808 + 0.0117271
[24]	cv_agg's auc: 0.510882 + 0.0112081
[25]	cv_agg's auc: 0.511072 + 0.0118836
[26]	

/home/wjunneng/Python/anaconda3/envs/lightgbm/lib/python3.6/site-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Parameter10', 'Parameter5', 'Parameter6', 'Parameter7', 'Parameter8', 'Parameter9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


In [124]:
params['n_estimators'] = len(adv_cv_results['auc-mean'])

model_adv = lgb.LGBMClassifier(**params)
model_adv.fit(df_adv.drop('Is_Test', axis=1), df_adv.loc[:, 'Is_Test'])

preds_adv = model_adv.predict_proba(df_adv.drop('Is_Test', axis=1))[:, 1]


In [125]:
preds_adv.shape


(7698,)

In [126]:
from sklearn.metrics import roc_auc_score, mean_squared_log_error

params = {
        'learning_rate': 0.01,
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'num_class': 4,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1,
        'max_depth': 7,
        'seed': 42,
        'num_leaves': 1000,
        'n_threads':10
    }

def run_cv(df_train, df_test, sample_weight=None):
    print(sample_weight)
    if sample_weight is not None:
        train_set = lgb.Dataset(
            df_train.drop('Quality_label', axis=1),
            label=df_train.loc[:, 'Quality_label'], weight=sample_weight)

    else:
        train_set = lgb.Dataset(
            df_train.drop('Quality_label', axis=1),
            label=df_train.loc[:, 'Quality_label'])

    # Perform cross validation with early stopping
    params.pop('n_estimators', None)
    
    N_FOLDS = 5
    cv_results = lgb.cv(
        params,
        train_set,
        num_boost_round=10000,
        nfold=N_FOLDS,
        categorical_feature=categorical_columns,
        early_stopping_rounds=200,
        verbose_eval=True,
        seed=42)
    
    print(cv_results)
    print('交叉验证中最优的multi_logloss-mean为 {:.5f}，对应的标准差为{:.5f}.'.format(
    cv_results['multi_logloss-mean'][-1], cv_results['multi_logloss-stdv'][-1]))
    
    print('模型最优的迭代次数为{}.'.format(len(cv_results['multi_logloss-mean'])))

    params['n_estimators'] = len(cv_results['multi_logloss-mean'])
    
    model_cv = lgb.LGBMClassifier(**params)
    model_cv.fit(df_train.drop('Quality_label', axis=1),
                 df_train.loc[:, 'Quality_label'])

    # AUC
    preds_test_cv = model_cv.predict_proba(
        df_test.drop('Quality_label', axis=1))[:, 1]
    auc_test_cv = mean_squared_log_error(df_test.loc[:, 'Quality_label'], preds_test_cv)
    print('模型在测试集上的效果是{:.5f}。'.format(
        auc_test_cv))

    return model_cv

model_cv = run_cv(X_train, X_valid)

None
[1]	cv_agg's multi_logloss: 1.28295 + 0.000539608
[2]	cv_agg's multi_logloss: 1.28043 + 0.00055615
[3]	cv_agg's multi_logloss: 1.27806 + 0.000633
[4]	cv_agg's multi_logloss: 1.27577 + 0.000653991
[5]	cv_agg's multi_logloss: 1.27357 + 0.000704831
[6]	cv_agg's multi_logloss: 1.27126 + 0.00076547
[7]	cv_agg's multi_logloss: 1.26882 + 0.000806752
[8]	cv_agg's multi_logloss: 1.2666 + 0.000899607
[9]	cv_agg's multi_logloss: 1.26429 + 0.00100225
[10]	cv_agg's multi_logloss: 1.2622 + 0.00105727
[11]	cv_agg's multi_logloss: 1.25999 + 0.00111027
[12]	cv_agg's multi_logloss: 1.25805 + 0.00120688
[13]	cv_agg's multi_logloss: 1.25585 + 0.00129493
[14]	cv_agg's multi_logloss: 1.25372 + 0.00148718
[15]	cv_agg's multi_logloss: 1.25164 + 0.00160326
[16]	cv_agg's multi_logloss: 1.24979 + 0.00169063
[17]	cv_agg's multi_logloss: 1.24787 + 0.00171518
[18]	cv_agg's multi_logloss: 1.24592 + 0.00175626
[19]	cv_agg's multi_logloss: 1.24416 + 0.00173776
[20]	cv_agg's multi_logloss: 1.2422 + 0.0017871
[21]	

/home/wjunneng/Python/anaconda3/envs/lightgbm/lib/python3.6/site-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Parameter10', 'Parameter5', 'Parameter6', 'Parameter7', 'Parameter8', 'Parameter9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


In [127]:
preds_adv[:len(X_train)]

array([0.30663527, 0.29104435, 0.30663527, ..., 0.30329901, 0.30329901,
       0.30663527])

In [128]:
model_cv_wight = run_cv(X_train, X_valid, sample_weight=preds_adv[:len(X_train)])

[0.30663527 0.29104435 0.30663527 ... 0.30329901 0.30329901 0.30663527]
[1]	cv_agg's multi_logloss: 1.28107 + 0.00104946
[2]	cv_agg's multi_logloss: 1.27878 + 0.0011632
[3]	cv_agg's multi_logloss: 1.27652 + 0.00132023
[4]	cv_agg's multi_logloss: 1.27432 + 0.0013996
[5]	cv_agg's multi_logloss: 1.27231 + 0.00140583
[6]	cv_agg's multi_logloss: 1.27009 + 0.00151348
[7]	cv_agg's multi_logloss: 1.2678 + 0.00152624
[8]	cv_agg's multi_logloss: 1.26569 + 0.00162266
[9]	cv_agg's multi_logloss: 1.26352 + 0.00171057
[10]	cv_agg's multi_logloss: 1.26146 + 0.00180955
[11]	cv_agg's multi_logloss: 1.25939 + 0.00188993
[12]	cv_agg's multi_logloss: 1.25756 + 0.00203081
[13]	cv_agg's multi_logloss: 1.2555 + 0.00215642
[14]	cv_agg's multi_logloss: 1.2535 + 0.00225602
[15]	cv_agg's multi_logloss: 1.25154 + 0.00240758
[16]	cv_agg's multi_logloss: 1.24981 + 0.00248886
[17]	cv_agg's multi_logloss: 1.24795 + 0.00254729
[18]	cv_agg's multi_logloss: 1.24606 + 0.00262592
[19]	cv_agg's multi_logloss: 1.24446 + 0.0

/home/wjunneng/Python/anaconda3/envs/lightgbm/lib/python3.6/site-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Parameter10', 'Parameter5', 'Parameter6', 'Parameter7', 'Parameter8', 'Parameter9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
